In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

In [ ]:
#  요기요 웹크롤링

In [ ]:
# 네이버/구글 이미지 검색 수집 (이미지, 날짜, 장소 등 가능한지)

In [11]:
from dotenv import load_dotenv
import os
import sys
import urllib.request
import urllib.parse
import pandas as pd
import json
load_dotenv()

True

` 방법 
1.네이버 이미지 관련 크롤링 
2.이미지 url, 게시일자 추출
3.제목,본문에서 <가게명,위치,가격 등 패턴 정규식 추출>
4.이미지다운로드
5.구조화된 데이터로 정리 (csv)
`

In [28]:
# 이미지 검색 
# 환경변수 키 불러오기
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')

# 검색어 인코딩
encText = urllib.parse.quote('서울 짜장면')
url = f"https://openapi.naver.com/v1/search/blog?query={encText}&display=100" # JSON 결과 

# Request 객체 생성 + 헤더 추가 
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)

# 응답 받기
with urllib.request.urlopen(request) as response:
    result = response.read().decode('utf-8')

# JSON 파싱
data = json.loads(result)
items = data['items']

# 리스트로 정리 
items_list = []
for item in items:
    items_list.append([
        item.get('title').replace('<b>','').replace('</b>',''),
        item.get('link'),
        item.get('description').replace('<b>','').replace('</b>',''),
        item.get('postdate')
    ])
    
# DataFrame 생성
seoul_black_noodle_100 = pd.DataFrame(items_list,columns=['title','link','postdate','description'])
soeul_black_noodle_100

# 데이터 프레임 저장
df.to_csv('data/test1_seoul_black_noodle_100.csv',encoding='utf-8-sig' ,index=False)

# 데이터 추출
# import json
# items = response.json()['items']

In [23]:
seoul_black_noodle_100.iloc[:,2:3]

,postdate
0,업체에 서비스를 제공받아 직접 방문하여 작성한 리뷰입니다 맛돌이들이라면 모두 아는 ...
1,잘 먹었습니다. 또 오겠습니다. 주관적 평점 5 #팔공 #중화요리팔공 #관악구맛집 ...
2,#광화문맛집 #서울광화문맛집 #광화문역맛집 #광화문주변맛집 #광화문역점심맛집 #경복...
3,"서촌 맛집 추천, 서울 경복궁 서촌마을 짜장면 중식 혼밥 맛있는 식당 서울 서촌 마..."
4,서울 짜장면 맛집! 서울 동대문 창신동 맛집! 전통 수타면 방식의 동대문 일품손짜장...
...,...
95,#짜장면 #간짜장 #짜장면맛집 #서울짜장면맛집 #중식당 #블랙데이 #짜장면데이 #홍...
96,"중화요리 생각날 때 맛있게 먹고 싶을 때 최근에 먹어본 짜장면, 짬뽕, 탕수육 서울..."
97,"채끝살 김치짜장면 서울시스터즈 김치짜장면, 채끝살, 트러플오일,양파 채끝살 김치짜장..."
98,구매는 여기서 https://www.rodemfoodshop.com/goods/go...
